# Saving and loading models

## Saving the model during development using checkpoints

[Source of the below script](https://www.tensorflow.org/tutorials/keras/save_and_load)

In [2]:
!pip install pyyaml h5py


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2023-03-09 17:28:02.189557: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.11.0


In [4]:
# Create a sample dataset 

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [5]:
# Define a simple model

# Define a simple sequential model
def create_model():
  model = tf.keras.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


2023-03-09 17:28:12.552450: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
checkpoint_path = "training_1/cp.ckpt" # Provide the checkpoint path here
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
31/32 [============================>.] - ETA: 0s - loss: 1.1588 - sparse_categorical_accuracy: 0.6694
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 10ms/step - loss: 1.1579 - sparse_categorical_accuracy: 0.6680 - val_loss: 0.7221 - val_sparse_categorical_accuracy: 0.7770
Epoch 2/10
29/32 [==========================>...] - ETA: 0s - loss: 0.4194 - sparse_categorical_accuracy: 0.8912
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 6ms/step - loss: 0.4141 - sparse_categorical_accuracy: 0.8950 - val_loss: 0.5458 - val_sparse_categorical_accuracy: 0.8250
Epoch 3/10
32/32 [==============================] - ETA: 0s - loss: 0.2768 - sparse_categorical_accuracy: 0.9280
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 6ms/step - loss: 0.2768 - sparse_categorical_accuracy: 0.9280 - val_loss: 0.4530 - val_sparse_categorical_accuracy: 0.8590
Epoch 4/10
30/32 [=========

In [7]:
os.listdir(checkpoint_dir) # To look at what files got generated

['cp.ckpt.data-00000-of-00001', 'checkpoint', 'cp.ckpt.index']

## What are these files?
---
The above code stores the weights to a collection of checkpoint-formatted files that contain only the trained weights in a binary format. Checkpoints contain:

One or more shards that contain your model's weights.
An index file that indicates which weights are stored in which shard.
If you are training a model on a single machine, you'll have one shard with the suffix: .data-00000-of-00001

## Loading the model from the saved checkpoints


In [8]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3564 - sparse_categorical_accuracy: 0.0930 - 105ms/epoch - 3ms/step
Untrained model, accuracy:  9.30%


In [9]:
# Callback checkpoints
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


In [10]:
# To obtain the latest checkpoint of training
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [11]:
# To load the model with latest checkpoint weights

# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4838 - sparse_categorical_accuracy: 0.8740 - 117ms/epoch - 4ms/step
Restored model, accuracy: 87.40%


## Saving Weights & Model Configuration Separately

In [12]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc)) 

32/32 - 0s - loss: 0.4838 - sparse_categorical_accuracy: 0.8740 - 124ms/epoch - 4ms/step
Restored model, accuracy: 87.40%


## Saving Complete Model

In [13]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 0s 3ms/step - loss: 1.2075 - sparse_categorical_accuracy: 0.6540
Epoch 2/5
32/32 [==============================] - 0s 3ms/step - loss: 0.4273 - sparse_categorical_accuracy: 0.8850
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2909 - sparse_categorical_accuracy: 0.9260
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2112 - sparse_categorical_accuracy: 0.9510
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1479 - sparse_categorical_accuracy: 0.9760


INFO:tensorflow:Assets written to: saved_model/my_model/assets


INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [14]:
# Loading from the saved model
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Saving & Loading model in HDF5 Format

In [15]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')

# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Epoch 1/5
32/32 [==============================] - 0s 3ms/step - loss: 1.1457 - sparse_categorical_accuracy: 0.6650
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4231 - sparse_categorical_accuracy: 0.8830
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2898 - sparse_categorical_accuracy: 0.9180
Epoch 4/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2097 - sparse_categorical_accuracy: 0.9540
Epoch 5/5
32/32 [==============================] - 0s 3ms/step - loss: 0.1607 - sparse_categorical_accuracy: 0.9660
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 d